In [1]:
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time

import datasets.mvtec as mvtec

from feature_extraction import Resnet18Features, concatenateLayers, extractEmbeddingVectors, extractEmbeddingVectorsDataloader
from distribution_fitting import getMeanCov
from distance_calculation import calculateScoreMaps
from padim_utils import saveFeatures



In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [3]:
class_name = 'bottle'
data_path = '../data/mvtec_dataset/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
x, y, mask = next(iter(dataloader))
x = x.to(device)

x.shape

torch.Size([32, 3, 224, 224])

In [4]:
x.device

device(type='cpu')

In [5]:
resnet18_features = Resnet18Features(device)

In [6]:
next(resnet18_features.resnet18.parameters()).is_cuda

False

In [7]:
tic = time.perf_counter()
batch_embedding_vectors = extractEmbeddingVectors(resnet18_features, x, device)
print(time.perf_counter()-tic)


0.9296788219944574


In [8]:
batch_embedding_vectors.shape

torch.Size([32, 448, 56, 56])

In [9]:
tic = time.perf_counter()
embedding_vectors = extractEmbeddingVectorsDataloader(resnet18_features, dataloader, device)
print(time.perf_counter()-tic)


Feature extraction: 100%|██████████| 7/7 [00:13<00:00,  1.86s/it]

13.023028719995636


In [10]:
embedding_vectors.shape

torch.Size([209, 448, 56, 56])

In [11]:
embedding_vectors = embedding_vectors.to(device)

In [12]:
tic = time.perf_counter()
mean, cov_inv = getMeanCov(embedding_vectors, torch.device('cpu'))
print(time.perf_counter()-tic)


Calculating mean


Calculating covariance: 100%|██████████| 3136/3136 [00:40<00:00, 77.19it/s]


Calculating inverse of covariance
53.72385025100084


In [15]:
mean.shape

torch.Size([448, 3136])

In [16]:
cov_inv.shape

torch.Size([448, 448, 3136])

In [17]:
torch.save(mean, './distributions/test_mean.pkl')
torch.save(cov_inv, './distributions/test_cov_inv.pkl')


In [18]:
PATH = './distributions/test.pkl'
saveFeatures((mean, cov_inv), PATH)

In [2]:
import pickle
file = open('./distributions/test2.pkl', 'wb')
pickle.dump((mean, cov_inv), file, protocol=-1)
file.close()

NameError: name 'mean' is not defined

In [21]:
# mean = mean.to(device)
# cov_inv = cov_inv.to(device)

In [22]:
tic = time.perf_counter()
score_map = calculateScoreMaps(mean, cov_inv, embedding_vectors)#[0].unsqueeze(0)
print(time.perf_counter()-tic)


Calculating mahalonobis distances:   0%|          | 0/209 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [17]:
score_map.shape

torch.Size([209, 56, 56])